### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
23,42038913,10454,79085,1001,2,76,1119,6,2024-12-20 21:59,2024-12-20 21:59,0,1,3100,5,1
24,42276528,47939,79085,51530,0,0,1120,0,2024-12-21 18:00,2024-12-21 18:00,0,2,2876,1,1
25,42084850,79866,79085,1349,1,69,1120,1,2024-12-22 21:59,2024-12-22 21:59,0,0,19453,5,1
26,42286191,123507,79085,51530,0,0,1120,2,2024-12-23 22:00,2024-12-23 22:00,0,2,3255,1,1
27,42286176,116548,79085,51530,0,0,1120,3,2024-12-25 01:30,2024-12-25 01:30,1,0,5424,3,1
28,42084860,79085,124139,1349,2,69,1120,4,2024-12-25 21:59,2024-12-25 21:59,0,2,14087,3,1
29,42286131,90417,79085,51530,0,0,1120,5,2024-12-26 15:00,2024-12-26 15:00,0,1,4110,6,1
30,42282927,142693,79085,1001,3,76,1120,6,2024-12-27 21:59,2024-12-27 21:59,0,1,3500,6,1
31,42286404,79085,47939,51530,0,0,1121,0,2024-12-28 18:00,2024-12-28 18:00,1,0,3085,6,1
32,42084872,78671,79085,1349,3,69,1121,1,2024-12-29 21:59,2024-12-30 00:05,2,0,17894,6,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

13

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,10454,46187,45189,1,7,2,0,0,4-2-2-2,7,7,6,42038913
1,79085,47197,48195,2,10,2,0,0,3-4-3BC,7,8,7,42038913
0,47939,23376,32181,4,7,3,1,0,Gut-Gut,11,9,8,42276528
1,79085,46809,38004,2,2,2,0,0,4-4-2c,8,9,10,42276528
0,79866,34662,37249,1,6,0,0,0,Haikyuu!,9,8,10,42084850
1,79085,35323,32736,0,6,0,0,0,4-3-3aRU,9,10,10,42084850
0,123507,48248,30886,0,2,4,3,1,futebol 2.0,7,9,6,42286191
1,79085,21718,39080,0,17,0,0,0,3-5-2aVG,9,10,9,42286191
0,116548,27640,36020,0,10,2,0,0,4-4-2,10,10,9,42286176
1,79085,42318,33938,3,6,4,0,0,4-3-3aRU,9,9,10,42286176


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1476

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,4,12,4,0,0,22,4,12518022004010,1349
1,89569,4,10,3,1,0,15,3,10512015003002,1349
2,79866,4,10,3,1,0,10,1,10509010003005,1349
3,6295,4,9,3,0,1,21,9,9512021003009,1349
4,122169,4,6,2,0,2,21,11,6510021002004,1349
5,23755,4,6,2,0,2,17,11,6506017002011,1349
6,78671,4,6,2,0,2,9,6,6503009002007,1349
7,124139,4,6,2,0,2,7,6,6501007002008,1349
8,48541,4,3,1,0,3,5,22,3483005001001,1349
9,79085,4,2,0,2,2,1,5,2496001000006,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()